# What's in an Avocado Toast: A Supply Chain Analysis

![](avocado_wallpaper.jpeg)

You find yourself in London, crafting a delectable avocado toast, a dish that has risen dramatically in popularity on breakfast menus since the 2010s. This straightforward recipe requires just five ingredients: a ripe avocado, half a lemon, a generous pinch of salt flakes, two slices of sourdough bread, and a good drizzle of extra virgin olive oil. Most of these ingredients are now staples in grocery stores, and as you will find with this project, that is no small feat!

In this project, you'll conduct a supply chain analysis of three ingredients used in avocado toast using the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin.
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple category tags in the `categories_tags` column.
There is a column in each CSV file called `origins_tags`, which contains strings for the country of origin of each item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

[Open Food Facts database](https://world.openfoodfacts.org/)

In [47]:
import pandas as pd
import numpy as np

## Creating the DataFrames
The first step of the project was to create the DataFrames for each key ingredient: avocados, olive_oil, and sourdough. The csv files all use tabs as separators, and the project instructions listed what columns to keep from the csvs when reading them in as DataFrames.

In [48]:
#Creating the DataFrame 'avocado' using the relevant columns
avocado = pd.read_csv('data/avocado.csv', sep = '\t', usecols = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands','brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins', 'origins_tags' ])
avocado.head()

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
0,0059749979702,fr,NaN,NaN,NaN,NaN,Naturalia,naturalia,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Canada,en:canada,NaN,NaN
1,7610095131409,en,NaN,NaN,NaN,NaN,Zweifel,zweifel,"en:snacks,en:salty-snacks,en:appetizers,en:chi...","en:vegetarian,en:vegan","Switzerland, World","en:switzerland,en:world",NaN,NaN
2,4005514005578,en,Gelbe Linse Avocado Brotaufstrich,NaN,NaN,NaN,Tartex,tartex,de:abendbrotsufstrich,"en:organic,en:eu-organic,en:eg-oko-verordnung",Germany,en:germany,NaN,NaN
3,0879890002513,en,Avocado toast chili lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN
4,0223086613685,en,Avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN


In [49]:
#Creating the DataFrame 'olive_oil' using the relevant columns
olive_oil = pd.read_csv('data/olive_oil.csv', sep = '\t', usecols = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands','brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins', 'origins_tags'])
olive_oil.head()

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
0,3560070910366,fr,NaN,1 l,NaN,"en:bottle,fr:bouteille-verre-de-6g,fr:bouteill...","Carrefour Bio,Carrefour","carrefour-bio,carrefour","en:plant-based-foods-and-beverages,en:plant-ba...","en:organic,en:eu-organic,en:non-eu-agriculture...","France, Pologne, Roumanie","en:france,en:poland,en:romania","Espagne, Tunisie","en:spain,en:tunisia"
1,3270190008262,fr,NaN,1 l,NaN,"en:glass,en:bottle,fr:bouchon-metallique,fr:bo...",Carrefour,carrefour,"en:plant-based-foods-and-beverages,en:plant-ba...",en:green-dot,"France, Guadeloupe, Roumanie","en:france,en:guadeloupe,en:romania",Union européenne,en:european-union
2,6191509903023,en,Huile d'olive vierge extra,1 l,NaN,"en:glass,en:bottle,fr:bouchon-metal",Terra Delyssa,terra-delyssa,"en:plant-based-foods-and-beverages,en:plant-ba...","en:organic,en:certified-by-ecocert,en:eu-organ...","France, United States","en:france,en:united-states",Tunisia,en:tunisia
3,6191509900855,fr,NaN,750 ml,NaN,"en:glass,en:bottle",Terra Delyssa,terra-delyssa,"en:plant-based-foods-and-beverages,en:plant-ba...","en:organic,en:certified-by-ecocert,en:eu-organ...","France, Suisse","en:france,en:switzerland",Tunisie,en:tunisia
4,3019081241599,fr,NaN,145 g,NaN,fr:boite-conserve-metal,Petit Navire,petit-navire,"en:seafood,en:canned-foods,en:fishes,en:fatty-...",en:no-artificial-flavors,France,en:france,"Zone FAO 27, Usine portugal","fr:zone-fao-27,fr:usine-portugal"


In [50]:
#Creating the DataFrame 'sourdough' using the relevant columns
sourdough = pd.read_csv('data/sourdough.csv', sep = '\t', usecols = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands','brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins', 'origins_tags'])
sourdough.head()

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
0,5000169636046,en,"Wood-fored Fennel Sausage, ‘Nduja Sourdough Pizza",NaN,NaN,NaN,NaN,NaN,NaN,NaN,United Kingdom,en:united-kingdom,NaN,NaN
1,0850026434323,en,Cinnamon Spouted Sourdough,25 oz,NaN,NaN,NaN,NaN,NaN,"en:vegetarian,en:no-artificial-flavors,en:vega...",United States,en:united-states,NaN,NaN
2,0237653602484,en,San Francisco sourdough,20.0 oz,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN
3,4056489462187,en,Chargrilled vegetable and basil pesto woodfire...,NaN,NaN,NaN,"Lidl,Deluxe","lidl,deluxe","en:meals,en:pizzas-pies-and-quiches,en:pizzas",NaN,Ireland,en:ireland,NaN,NaN
4,10500016941075200179,fr,NaN,NaN,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,France,en:france,NaN,NaN


## Reading in the "Relevant Categories"
The next step was reading in the relevant category tags for each key ingredient. These tags were provided in txt files and were read in as lists to make filtering for them easier down the line. 

In [51]:
#Creating a list from the relevant_avocado_categories txt file
with open('data/relevant_avocado_categories.txt', 'r') as file:
    avocado_categories=[line.strip('\n') for line in file]
print(avocado_categories)

['en:avocadoes', 'en:avocados', 'en:fresh-foods', 'en:fresh-vegetables', 'en:fruchte', 'en:fruits', 'en:raw-green-avocados', 'en:tropical-fruits', 'en:tropische-fruchte', 'en:vegetables-based-foods', 'fr:hass-avocados']


In [52]:
#Creating a list from the relevant_olive_oil_categories txt file
with open('data/relevant_olive_oil_categories.txt', 'r') as file:
    olive_oil_categories = [line.strip('\n') for line in file]
print(olive_oil_categories)

['ar:huile-d-olive', 'ar:oil', 'bg:green-olive-paste', 'de:ol', 'en:aceites-de-oliva', 'en:aceites-de-oliva-virgen', 'en:aceites-de-oliva-virgen-extra', 'en:aceites-vegetales', 'en:aceites-y-grasas', 'en:aromatised-olive-oils', 'en:blend-of-olive-oils', 'en:combined-oil', 'en:combined-oil-blended-vegetable-oils', 'en:cooking-oil', 'en:extra-virgin-olive-oils', 'en:extra-virgin-olive-oils-with-garlic', 'en:extra-virgin-olive-oils-with-truffles', 'en:falvoured-olive-oils', 'en:flavored-oils', 'en:flavoured-extra-virgin-olive-oils', 'en:flavoured-oils', 'en:flavoured-olive-oil-sprays', 'en:flavoured-olive-oils', 'en:fruit-and-fruit-seed-oils', 'en:greek-cretan-kolymbari-region-olive-oil', 'en:greek-oils-and-fats', 'en:huile-d-olive-vierge-extra', 'en:huiles', 'en:huiles-d-olive', 'en:huiles-d-olive-vierges', 'en:huiles-d-olive-vierges-extra', 'en:mixed-oils', 'en:mixed-vegetable-oils', 'en:mixed-vegetables', 'en:oil', 'en:oil-sprays', 'en:oilive-oil-sprays', 'en:oils', 'en:olive-oi', 'en:

In [53]:
#Create a list of the relevant_sourdough_categories.txt file
with open('data/relevant_sourdough_categories.txt', 'r') as file: 
    sourdough_categories = [line.strip('\n') for line in file]
print(sourdough_categories)

['en:bagel-breads', 'en:baguettes', 'en:bakery-products', 'en:bran-bread', 'en:breads', 'en:buns', 'en:confectioneries', 'en:crackers', 'en:crackers-with-natural-sourdough', 'en:crackers-with-wholegrain-rye', 'en:crispbreads', 'en:english-muffins', 'en:flatbreads', 'en:garlic-breads', 'en:gluten-free-breads', 'en:olive-breads', 'en:panini-breads', 'en:pastries', 'en:pre-baked-breads', 'en:rye-and-wheat-breads', 'en:rye-breads', 'en:sliced-breads', 'en:sordough-breads', 'en:sourdough', 'en:sourdough-bread', 'en:sourdough-breads', 'en:sourdough-breads-with-rosemary', 'en:sourdough-pita-bread', 'en:special-breads', 'en:wheat-breads', 'en:wheat-flatbreads', 'en:wholemeal-breads', 'en:wholemeal-sliced-breads']


## Organizing the "categories_tags" column
In each DataFrame, each entry in the "categories_tags" column was turned into a list to make filtering easier. All rows with null values in that column were dropped. The lengths of the DataFrames were checked to compare to later. 

In [54]:
#Create lists for each row in the 'categories_tags' column by splitting up the tags
avocado['categories_tags'] = avocado['categories_tags'].str.split(',')
olive_oil['categories_tags'] = olive_oil['categories_tags'].str.split(',')
sourdough['categories_tags'] = sourdough['categories_tags'].str.split(',')

#Drop all null values in the 'categories_tags' column
avocado = avocado.dropna(subset = 'categories_tags')
olive_oil = olive_oil.dropna(subset = 'categories_tags')
sourdough = sourdough.dropna(subset = 'categories_tags')
print(len(avocado), len(olive_oil), len(sourdough))
avocado.head()

752 6630 558


,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
0,0059749979702,fr,NaN,NaN,NaN,NaN,Naturalia,naturalia,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,Canada,en:canada,NaN,NaN
1,7610095131409,en,NaN,NaN,NaN,NaN,Zweifel,zweifel,"[en:snacks, en:salty-snacks, en:appetizers, en...","en:vegetarian,en:vegan","Switzerland, World","en:switzerland,en:world",NaN,NaN
2,4005514005578,en,Gelbe Linse Avocado Brotaufstrich,NaN,NaN,NaN,Tartex,tartex,[de:abendbrotsufstrich],"en:organic,en:eu-organic,en:eg-oko-verordnung",Germany,en:germany,NaN,NaN
5,3662994002063,fr,NaN,3 fruits,NaN,NaN,la compagnie des fruits mûrs,la-compagnie-des-fruits-murs,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,France,en:france,NaN,NaN
6,8437013031011,fr,NaN,1 kg,NaN,NaN,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,France,en:france,NaN,NaN


## Filtering "categories_tags" column
The "categories_tags" column in each DataFrame was then filtered to check if each entry contained at least one of the relevant category tags for each respective ingredient. Rows that did not contain at least one relevant category tag were dropped. The lengths of the DataFrames were checked again to see how many rows were dropped due to not containing a relevant category tag. 

In [55]:
#Subset to keep only the rows that contain at least one of the tags from avocado_categories in their categories_tags column
avocado = avocado[avocado['categories_tags'].apply(lambda x: any([i for i in x if i in avocado_categories]))]
olive_oil = olive_oil[olive_oil['categories_tags'].apply(lambda x: any([i for i in x if i in olive_oil_categories]))]
sourdough = sourdough[sourdough['categories_tags'].apply(lambda x: any([i for i in x if i in sourdough_categories]))]
print(len(avocado), len(olive_oil), len(sourdough))    

182 5290 399


## Finding the most common country of origin for each ingredient in the UK
New DataFrames for each ingredient were created that contained only ingredients in the UK. The number of times each country of origin appeared was then counted and saved as pandas Series. The Series were converted to DataFrames, and the top country of origin for each ingredient was indexed out. The names of the top countries of origin were then cleaned up to match the instructions (only contain letters and spaces with no hyphens or other characters). From this data, Peru is the main exporter of avocados to the UK, Greece is the main exporter of olive oil to the UK, and the UK gets most of its sourdough bread from within its own borders. 

In [56]:
#Create a subset of the list for avocados in the UK
avocado_uk = avocado[avocado['countries'] == 'United Kingdom']

#Count how many times each country of origin appears on the list
avocado_uk_origin_count = avocado_uk['origins_tags'].value_counts()
print(avocado_uk_origin_count)

#Find the top avocado originator for avocados in the UK
top_avocado_origin = pd.DataFrame(avocado_uk_origin_count).reset_index().iloc[0,0]
print(top_avocado_origin)

#Clean up the name and ensure it doesn't have any whitespace
top_avocado_origin = top_avocado_origin.strip('en:')
print(top_avocado_origin)
print(len(top_avocado_origin))

en:peru             2
en:spain,en:peru    1
en:chile,en:peru    1
en:israel           1
Name: origins_tags, dtype: int64
en:peru
peru
4


In [57]:
#Create a subset of the list for olive_oils in the UK
olive_oil_uk = olive_oil[olive_oil['countries'] == 'United Kingdom']

#Count how many times each country of origin appears on the list
olive_oil_uk_origin_count = olive_oil_uk['origins_tags'].value_counts()
print(olive_oil_uk_origin_count)

#Find the top olive oil originator for olive oil in the UK
top_olive_oil_origin = pd.DataFrame(olive_oil_uk_origin_count).reset_index().iloc[0,0]
print(top_olive_oil_origin)

#Clean up the name and ensure it doesn't have any whitespace
top_olive_oil_origin = top_olive_oil_origin.replace('en:', '')
print(top_olive_oil_origin)
print(len(top_olive_oil_origin))

en:greece                                             6
en:spain                                              4
en:italy                                              4
en:greece,en:italy,en:portugal,en:spain,en:tunisia    2
en:produce-of-italy                                   1
en:european-union-and-non-european-union              1
en:produced-in-italy                                  1
en:european-union                                     1
Name: origins_tags, dtype: int64
en:greece
greece
6


In [58]:
#Create a subset of the list for sourdough in the UK
sourdough_uk = sourdough[sourdough['countries'] == 'United Kingdom']

#Count how many times each country of origin appears on the list
sourdough_uk_origin_count = sourdough_uk['origins_tags'].value_counts()
print(sourdough_uk_origin_count)

#Find the top sourdough originator for sourdough in the UK
top_sourdough_origin = pd.DataFrame(sourdough_uk_origin_count).reset_index().iloc[0,0]
print(top_sourdough_origin)

#Clean up the name and ensure it doesn't have any whitespace
top_sourdough_origin = top_sourdough_origin.replace('en:', '')
top_sourdough_origin = top_sourdough_origin.replace('-', ' ')
print(top_sourdough_origin)
print(len(top_sourdough_origin))

en:united-kingdom    3
en:france            1
Name: origins_tags, dtype: int64
en:united-kingdom
united kingdom
14
